# Возврат исходных документов

В этом разделе показано, как доработать приложение, созданное в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart), чтобы оно возвращало документы, использованные для генерации.

Такое поведение поможет пользователям ознакомиться с источниками данных.

## Установка зависимостей

Для разработки используются модели генерации и эмбеддингов GigaChat, а также векторное хранилище Chroma.
Вы можете использовать любое [векторное хранилище](/docs/modules/data_connection/vectorstores/) или [ретривер](/docs/modules/data_connection/retrievers/)

Установите пакеты с помощью команды:

In [ ]:
%pip install --upgrade --quiet  gigachain langchainhub chromadb bs4

## Цепочка без возврата исходных документов

Пример цепочки, которая не возвращает исходные документы. Пример разобран в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart):

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.chat_models import GigaChat
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# Загрузка, разделение на части и индексация содержимого блога.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)

# Извлечение данных и генерация с помощью релевантных фрагментов блога.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [3]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, simpler steps. This allows an agent to better understand and plan ahead for the task at hand. In the context provided, it is mentioned that the AI assistant can parse user input into several tasks, each with its own unique identifier and dependencies on previous tasks. This helps in breaking down complex tasks into manageable parts, making it easier for the agent to complete the task successfully.'

## Добавление исходных документов {#adding-sources}

LCEL позволяет возвращать документы, полученные ретривером:

In [4]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is Task Decomposition")

{'context': [Document(page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the path of the user-mentioned resources for your task planning.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Fi

:::{.callout-tip}

Check out the [LangSmith trace](https://smith.langchain.com/public/007d7e01-cb62-4a84-8b71-b24767f953ee/r)

:::